In [1]:
pip install netCDF4

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [91]:
pip install pyhomogeneity

Note: you may need to restart the kernel to use updated packages.


In [92]:
import netCDF4
import matplotlib.pyplot as plt
import datetime 
import statsmodels as statsmodels
import statistics as stats
import decimal
import os
import numpy as np
import copy
import pyhomogeneity as hg

In [68]:


class ReadData:
    """
    ReadData-- generates the dataset
    
    TODO: currently doesn't use generators, so it can be slow.
    """
    DMI_data = {}
    DMI_list = [] #sorted version of the dictionary
    sortedKeys_DMI = []
    TRMM_data = {}
    TRMM_list = []
    TRMM_acc_list = [] #weekly precepitation
    TRMM_mod7 = 6 #this is the only way to get the data to match up with NINO and DMI
    sortedKeys_TRMM = []
    ElNino_data= {}
    ElNino_list = [[],[],[],[]]
    sortedKeys_Nino = [[],[],[],[]] #currently, we have 4 Nino items
    data_dict = {}
    
    #constants
    begin = datetime.date(1998, 1, 1) #begin of dates we look at. We need to start a bit late so we can get 7 previous days to use for acc
    end = datetime.date(2019, 12, 25) #end of dates we look at
    #note that we have to discard this final point to make everything even
    
    
    
    
    def __init__(self):
        
        #in the init phase, we generated each data set and sort the dictionary keys
        
        self.DMI_data = self.generateDataDMI()
        print("DMI data generated")
        self.sortedKeys_DMI = self.sortKeys(self.DMI_data)   
        for i in self.sortedKeys_DMI:
            self.DMI_list.append(self.DMI_data[i])
        
        self.TRMM_data = self.generateDataTrim()
        self.sortedKeys_TRMM = self.sortKeys(self.TRMM_data)  
        for i in self.sortedKeys_TRMM:
            self.TRMM_list.append(self.TRMM_data[i])
        print("TRMM data generated")
        sum_L = self.accumulateTRMM()
        self.TRMM_acc_list = self.takeModN_TRMM(sum_L) #this creates a list with the summed precipitation fo the previous days
        
        #and we only take days that are mod7 == 0
        #
        
        
        
        self.ElNino_data = self.generateDataElNino()
        for i in range(0,len(self.ElNino_data)):
            self.sortedKeys_Nino[i] = self.sortKeys(self.ElNino_data[i])
            for j in self.sortedKeys_Nino[i]:
                self.ElNino_list[i].append(self.ElNino_data[i][j])
                #print(j)
        print("ELNino data generated")
        
    def accumulateTRMM(self):
        TRMM_list_update = [] #because we have to skip first date, we lose one data point
        for i in range(6, len(self.TRMM_list)):
            TRMM_list_update.append(copy.deepcopy(self.TRMM_list[i]) ) #arrays are pass by REFRENCE--need to copy.
            for j in range(0,6):
                TRMM_list_update[i-6] += self.TRMM_list[i-j-1]
        return TRMM_list_update
    def takeModN_TRMM(self,L):
        TRMM_list_res = []
        for i in range(0,len(L)):
            if(i % 7 == self.TRMM_mod7): #only take number with mod of TRMM_mod7
                TRMM_list_res.append(L[i])
                #look at this sorted key, why are we missing the last key?
            
        return TRMM_list_res
    
        
    def sortKeys(self,data):
        return sorted(data,key = lambda key: self.toDateTime(key )-self.begin)
    def findMinData(self,data):
        v = 0
        for key in data.keys():
            diff = self.toDateTime(key )-self.begin
            if(v == 0 or v> diff):
                v = diff
        print(v)
        return v
        
        
    
    """
    Generate DMI dataset--used for (???)
    """
    def generateDataDMI(self):
        iodfile = "dmi.nc" #were reading the DMI file
        f = netCDF4.Dataset(iodfile, 'r')
        SSTAnp=f.variables['DMI'][:]
        weeknp=f.variables['WEDCEN2'][:]
        SSTA=SSTAnp.tolist() #list of DMI SSTA
        week=weeknp.tolist() 
        f.close()
        DMIdate=[]     #DATE OF DMI 
        for i in range(0, len(week)):    
            days=week[i]
            start=datetime.date(1900, 1, 1)
            delta=datetime.timedelta(days)
            offset=start+delta
            DMIdate.append(offset)

        date_DMI={}
        skip = True
        for j in range(0, len(SSTA)): #skip first data point, so we can meet with TRMM
            if(self.isInTimeRange(DMIdate[j])):
                if(not skip):
                    date_DMI[str(DMIdate[j]).replace("-","")] =  SSTA[j]
                else:
                    skip = False #skip one data point
            else:
                pass
                #this data we will just discard for now
        #print("len",len(date_DMI)) 1146 length, since we skip one data point
        return date_DMI
    
    def generateDataElNino(self):
        ninoindicesfile="elnino indices.txt"
        f=open(ninoindicesfile, "r")
        linelist=list(f.readlines())    
        #### ARRAYS OF FORMAT (DATETIME, DMI)
        date_Ninos=[{},{},{},{}]
        #12,3,34,4 are the ninos we take
        
        search_range = [[19,23],[32,36],[45,49],[58,62]]
        
        for j in range(0,len(search_range)):
            skip = True
            for i in range(4, len(linelist)): #skip the first datapoint
                date_str = self.replaceDateTime(linelist[i][1:10])
                if (self.isInTimeRangeStr(date_str)):
                    if(not skip):
                        date_Ninos[j][date_str] =  float(linelist[i][search_range[j][0]:search_range[j][1]])            
                    else:
                        skip = False
                else:
                    pass
                    #this is the date we discard
            print(len(date_Ninos[j]) )  #is 1146, since we skip one point
        f.close() 

        
        
        return date_Ninos
    """
    Generate TRMM dataset
    """
    def generateDataTrim(self): #this is very slow, potentially increase speed at some point
        TRMMfolder="TRMM1/"
        datetime_prec={} #create a dictionary with the dates
        for filename in os.listdir(TRMMfolder):
            if filename.endswith('nc4'):
                f=netCDF4.Dataset(TRMMfolder + str(filename))
                precp=f.variables['precipitation'][:]
                date_str = str(filename[11:19])
                #if(self.isInTimeRangeStr(date_str) and not str(self.toDateTime(date_str)-self.begin)=="0:00:00" and int(str(self.toDateTime(date_str)-self.begin).split(" ")[0]) % 7 == self.TRMM_mod7  ):
                if(self.isInTimeRangeStr(date_str)):    
                    datetime_prec[date_str] =  precp
                else:
                    pass
                f.close()
            else:
                pass
        
        #datetime_prec=np.array(datetime_prec)
        return datetime_prec
    def isInTimeRange(self,time):
        if (time >= self.begin and time <= self.end):
            return True
        return False
    def toDateTime(self,time):
        return datetime.date(int(time[0:4]),int(time[4:6]),int(time[6:8]))
    def isInTimeRangeStr(self,time):
        return self.isInTimeRange(self.toDateTime(time))
        
        
    def replaceDateTime(self,string):
        return str(datetime.datetime.strptime(string, '%d%b%Y')).split(" ")[0].replace("-","")
    
    
        
    
    
    





reader = ReadData()
    









DMI data generated
TRMM data generated
1146
1146
1146
1146
ELNino data generated


In [109]:
class AnalyzeData:
    "class for anaylzing the data we generated from a reader"
    reader_class = None
    snhtsetDMI = []
    snhtsetNino = []
    snhtsetTRMM = []
    mx_lon = 13
    mx_lat = 11
    def __init__(self,reader):
        self.reader_class = reader
        #self.generateAllSNHT()
    def generateAllSNHT(self):
        self.snhtsetDMI = self.SNHT(reader.DMI_list)
        
        print("dmi",type(reader.DMI_list[0]))
        self.snhtsetTRMM= [] #the snht set is ordered as a list (lon,lat,?)
        for lon in range(0,self.mx_lon):
            lat_list = []
            for lat in range(0,self.mx_lat):
                #print("trmm",type(self.getDatasetTRMM(lon,lat)[0]))
                lat_list.append(self.SNHT(self.getDatasetTRMM(lon,lat)))
                pass
            self.snhtsetTRMM.append(lat_list)
        
        
        self.snhtsetNino = []
        for i in reader.ElNino_list:    
            self.snhtsetNino.append(self.SNHT(i))
        #TODO: continue with taking the snhtset
    def getDatasetTRMM(self,lon,lat):
        #right now, we don't check to ensure the list is correctly sized, add that later
        #i,j means for each item, take the ith item, and the jth item from that list, and form it into a dataset
        list_res = []
        
        for item in self.reader_class.TRMM_acc_list:
            list_res.append(item[lon][lat])
        
        return list_res
    #(Old snht test)
    
    def SNHTALL(self,dataset):   #SNHT FOR time series of dimension 1 x n
        #This function is fairly slow, but we only need to run it once, so speed isn't all that important
        n=len(dataset)
        
        print(n)
        print(np.std(dataset) * ((n-1)/n))
        
        sd=np.std(dataset) #the previous pstvdev doesn't work for TRMM, plus is slightly wrong
        
        mean = stats.mean(dataset)
        #we want the mean of the first y years, not the total mean
        snhtset=[]
        for y in range(1,n-1): #Loops from Week 1 to week n-1
            summ1=0 
            for i in range(0, y): 
                summ1 += (dataset[i])-mean
            z_1= (1.0/(y*sd)) * (summ1)
            summ2=0
            for i in range(y, n):
                summ2 += (dataset[i]-mean)
            z_2= (1/( (n-y)*sd)) * (summ2)
            snhtset.append(y * (z_1 **2) + (n-y)*(z_2**2))
            print(y * (z_1 **2) + (n-y)*(z_2**2))
        return snhtset
    
    def SNHT(self,dataset):
        result = hg.snht_test(data)
        print(result)
        return hg.snht_test(data)

    def Pearlson(self,x, y, lag):      #LAG means that we compare x_t against y_{t-lag} where lag is positive
        x=x[lag:]
        y=y[:-lag]
        x_mean=stats.mean(x)
        y_mean=stats.mean(y)
        top=0
        n=len(x)-lag
        for j in range(0, n):
            top=top+(x[j]-x_mean)*(y[j]-y_mean)
        pearlson= top/(n*stats.stdev(x) * stats.stdev(y))   
        return pearlson

analyze = AnalyzeData(reader) 

#analyze.generateAllSNHT()

In [110]:
analyze.SNHTALL(analyze.reader_class.DMI_list)

1146
0.5634493072070124
1.988490374061122
3.2099628039167136
2.477641849885092
3.1842007653172164
1.8997497233380218
2.990806131062042
3.503342219140689
3.6260321598884313
2.841214111387945
1.703226582077105
1.1222997117298479
1.1629050079261156
0.9743738860203331
0.5132304369126559
0.3851245444962712
0.4046260517411164
0.5579417653806055
0.6392050910720595
0.47294841610658767
0.2680107385387682
0.17105158066022452
0.30412288543387767
0.29263477343761435
0.21281214485685093
0.04879171299075648
0.0017307252280080263
0.11311571585437132
0.44647223377024114
0.9783192652698305
1.3678417606791347
2.3055854039441748
3.2715887483477784
4.278267201365179
5.06113878109317
6.100016044608162
7.386550188034055
8.652269427005962
10.739591712767412
12.361873880304293
15.706610615324236
17.018486608702666
18.781072861348374
19.65683639101546
22.021314122196692
23.599487388311456
25.952594747440752
27.133478865028874
29.142248173761313
29.484954214987166
30.595723120556087
31.16936291341829
30.9777428

40.22161509567461
38.63109500211631
36.88046723519345
35.825317455698375
35.3400836136854
33.459335090947306
32.13362475441159
31.413519967641477
30.80948918627205
29.50203220363799
28.08999375015364
27.478070827603027
27.202526280905133
27.442442443296567
27.861608446043
27.913594336175315
28.099415626790744
28.9244167644468
30.136316087943538
31.520647740318214
32.872441224545696
33.788875367192
35.139668312307116
36.02618377367301
37.19562361641709
38.25470018671118
38.670207396345226
39.14005536542618
39.46727289048359
39.62213565907706
40.63116917362718
41.04013580985052
41.412328046088795
42.16351436849296
42.072901598364524
42.156248457124676
42.14366617093717
43.51874298907414
44.503130596140615
45.888475140771035
47.88501798019671
49.656760955091045
51.425767675501405
53.66768240531227
57.001737895677124
60.648716702745716
63.493810693841155
66.80513681370371
69.14550050452517
71.0657853686632
72.50099130480956
75.08380062115866
79.1299577558247
83.79208829788308
87.5277335185

[1.988490374061122,
 3.2099628039167136,
 2.477641849885092,
 3.1842007653172164,
 1.8997497233380218,
 2.990806131062042,
 3.503342219140689,
 3.6260321598884313,
 2.841214111387945,
 1.703226582077105,
 1.1222997117298479,
 1.1629050079261156,
 0.9743738860203331,
 0.5132304369126559,
 0.3851245444962712,
 0.4046260517411164,
 0.5579417653806055,
 0.6392050910720595,
 0.47294841610658767,
 0.2680107385387682,
 0.17105158066022452,
 0.30412288543387767,
 0.29263477343761435,
 0.21281214485685093,
 0.04879171299075648,
 0.0017307252280080263,
 0.11311571585437132,
 0.44647223377024114,
 0.9783192652698305,
 1.3678417606791347,
 2.3055854039441748,
 3.2715887483477784,
 4.278267201365179,
 5.06113878109317,
 6.100016044608162,
 7.386550188034055,
 8.652269427005962,
 10.739591712767412,
 12.361873880304293,
 15.706610615324236,
 17.018486608702666,
 18.781072861348374,
 19.65683639101546,
 22.021314122196692,
 23.599487388311456,
 25.952594747440752,
 27.133478865028874,
 29.14224817376

In [61]:
len(reader.TRMM_data['20090313'][0]) #its shaped at (n,13,11). 
print(len(reader.TRMM_acc_list))
print(len(reader.DMI_list))


1146
1147


In [59]:
print(reader.sortedKeys_TRMM)


['19980101', '19980102', '19980103', '19980104', '19980105', '19980106', '19980107', '19980108', '19980109', '19980110', '19980111', '19980112', '19980113', '19980114', '19980115', '19980116', '19980117', '19980118', '19980119', '19980120', '19980121', '19980122', '19980123', '19980124', '19980125', '19980126', '19980127', '19980128', '19980129', '19980130', '19980131', '19980201', '19980202', '19980203', '19980204', '19980205', '19980206', '19980207', '19980208', '19980209', '19980210', '19980211', '19980212', '19980213', '19980214', '19980215', '19980216', '19980217', '19980218', '19980219', '19980220', '19980221', '19980222', '19980223', '19980224', '19980225', '19980226', '19980227', '19980228', '19980301', '19980302', '19980303', '19980304', '19980305', '19980306', '19980307', '19980308', '19980309', '19980310', '19980311', '19980312', '19980313', '19980314', '19980315', '19980316', '19980317', '19980318', '19980319', '19980320', '19980321', '19980322', '19980323', '19980324', '19

In [60]:
print(reader.sortedKeys_DMI)

['19980107', '19980114', '19980121', '19980128', '19980204', '19980211', '19980218', '19980225', '19980304', '19980311', '19980318', '19980325', '19980401', '19980408', '19980415', '19980422', '19980429', '19980506', '19980513', '19980520', '19980527', '19980603', '19980610', '19980617', '19980624', '19980701', '19980708', '19980715', '19980722', '19980729', '19980805', '19980812', '19980819', '19980826', '19980902', '19980909', '19980916', '19980923', '19980930', '19981007', '19981014', '19981021', '19981028', '19981104', '19981111', '19981118', '19981125', '19981202', '19981209', '19981216', '19981223', '19981230', '19990106', '19990113', '19990120', '19990127', '19990203', '19990210', '19990217', '19990224', '19990303', '19990310', '19990317', '19990324', '19990331', '19990407', '19990414', '19990421', '19990428', '19990505', '19990512', '19990519', '19990526', '19990602', '19990609', '19990616', '19990623', '19990630', '19990707', '19990714', '19990721', '19990728', '19990804', '19